## Visualization of simulations

In [1]:
import os, sys
from polychrom.hdf5_format import list_URIs, load_URI, load_hdf5_file
from nglutils import nglutils as ngu
import numpy as np

In [2]:
folder = '2019-09-18_18:46_locus_pulling_first_tries'
#folder = 'generic_trajectory'
base_folder = '/net/levsha/share/simongh/sims/'
folder = os.path.join(base_folder, folder)
files = list_URIs(folder)

In [3]:
data = [load_hdf5_file(os.path.join(folder, 'starting_conformation_0.h5'))['pos']]
for file in files:
    data.append(load_URI(file)['pos'])
data = np.array(data)

### Show compartments

In [4]:
def mccmap(color):
    def cmap(val):
        return color
    return cmap

frames = slice(None, None)
show = slice(0, None) # A, B, C compartments
#info = load_hdf5_file(os.path.join(base_folder, 'generic_trajectory/final_conformation_0.h5'))
info = load_hdf5_file(os.path.join(folder, 'final_conformation_0.h5'))

chainlengths = np.bincount(info['c5_compartmentIDs'])
nframes = len(range(*frames.indices(data.shape[0])))
chains = [np.empty((nframes, chainlengths[0], 3)),
          np.empty((nframes, chainlengths[1], 3)),
          np.empty((nframes, chainlengths[2], 3))]
ignores = [[], [], []]
chaincount = [0, 0, 0]
curID = 0
for i, ID in enumerate(info['c5_compartmentIDs']):
    if not ID == curID:
        ignores[curID].append(chaincount[curID])
        curID = ID
    chains[ID][:, chaincount[ID], :] = data[frames, i, :]
    chaincount[ID] += 1

for i in range(len(ignores)):
    ignores[i] = np.array(ignores[i])

cmaps = [mccmap((1.0, 0.8, 0.8)), mccmap((0.8, 0.8, 1.0)), mccmap((1.0, 1.0, 0))]

view = ngu.xyz2nglview(chains[show], cmaps[show], exclude_bonds=ignores[show])
view

NGLWidget(count=101)

### Show chromosomes

In [5]:
# Load the final_configuration, which has chains and compartments with it, and display.
def mccmap(color):
    def cmap(val):
        return color
    return cmap

frames = slice(None, None)
show = slice(0, None) # show all chromosomes
#info = load_hdf5_file(os.path.join(base_folder, 'generic_trajectory/final_conformation_0.h5'))
info = load_hdf5_file(os.path.join(folder, 'final_conformation_0.h5'))

chains = []
for ch in info['chains']:
    chains.append(data[frames, ch[0]:ch[1], :])

cmaps = [mccmap((1.0, 0, 0)), mccmap((0, 1.0, 0)), mccmap((0, 0, 1.0)), mccmap((1.0, 1.0, 0))]

view = ngu.xyz2nglview(chains[show], cmaps[show])
view

NGLWidget(count=101)